In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from datasets import load_dataset
from accelerate import Accelerator
import numpy as np
import torch
from tqdm import tqdm
import argparse
import json
import time
import os
import sys
import re

import datasets

/home/intern/miniconda3/envs/hoang/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

tokenizer.add_special_tokens({'pad_token': '[PAD]'})


1

In [3]:

def tokenize(sample):
        # question = sample['conversations'][0]['content'].split("Step 1")[0]
        question = sample['prompt']
        samples = []
        for id, ans in enumerate(sample['answers']):
            sam = {}
            # ans = "Step 1" + sample['conversations'][0]['content'].split("Step 1")[1]
            message = [
                {"role":"user", "content":question},
                {"role":"assistant", "content":ans}
            ]
            sam['positive'] = tokenizer.apply_chat_template(
                # message, tokenize=False, add_generation_prompt=False).replace(tokenizer.bos_token, "")
                message, tokenize=False, add_generation_prompt=False)
            tokenized_pos = tokenizer(sam['positive'], truncation=False)
            sam["input_ids_j"] = tokenized_pos["input_ids"]
            sam["attention_mask_j"] = tokenized_pos["attention_mask"]
            if sample['label'][id] == 1:
                sam['label'] = 1
            elif sample['label'][id] == 0:
                sam['label'] = 0
            else:
                assert 1 == 0
            samples.append(sam)
        return samples
    

In [4]:
train_path = "RLHFlow/Mistral-MATH500-Test"

In [5]:
ds = load_dataset(train_path, split="train").shuffle(seed=42)
#ds = ds.select(range(2000))
# ds = ds.map(tokenize, num_proc=8)




In [6]:
new_ds = []
for sam in tqdm(ds):
    new_ds.extend(tokenize(sam))
print(len(new_ds))

100%|██████████| 500/500 [05:43<00:00,  1.46it/s]

512000


In [7]:
new_dataset = datasets.Dataset.from_list(new_ds)


In [10]:
from huggingface_hub import HfApi, HfFolder, Repository
from credential import HUGGINGFACE_TOKEN
# repo_name = "Colder203/Mistral-MATH500-Test"
# api = HfApi()
# repo_url = api.create_repo(repo_name, token=HUGGINGFACE_TOKEN)
#upload dataset
new_dataset.push_to_hub('Colder203/math500_test_llama', token=HUGGINGFACE_TOKEN)

Uploading the dataset shards: 100%|██████████| 7/7 [00:37<00:00,  5.39s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Colder203/math500_test_llama/commit/eb3e0c88002c8858cc2f65a2271881e051ec7760', commit_message='Upload dataset', commit_description='', oid='eb3e0c88002c8858cc2f65a2271881e051ec7760', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Colder203/math500_test_llama', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Colder203/math500_test_llama'), pr_revision=None, pr_num=None)

In [3]:
#load dataset
import datasets
new_ds = datasets.load_dataset('Colder203/math_test')

In [ ]:
from peft import PeftModel
from transformers import AutoModelForSequenceClassification

# base_model = AutoModelForSequenceClassification.from_pretrained("microsoft/Phi-3.5-mini-instruct")
base_model = AutoModelForSequenceClassification.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", num_labels = 1, torch_dtype=torch.float16, device_map={'': torch.cuda.current_device()})
base_model.config.use_cache = False
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.resize_token_embeddings(lens(tokenizer))

# Load the model with ignore_mismatched_sizes=True
model = PeftModel.from_pretrained(base_model, "Colder203/llama3b_orm" 
                                #   ignore_mismatched_sizes=True
                                  )


Loading checkpoint shards:  50%|█████     | 1/2 [00:08<00:08,  8.88s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 11.75 GiB of which 7.12 MiB is free. Process 1089242 has 6.29 GiB memory in use. Including non-PyTorch memory, this process has 5.35 GiB memory in use. Of the allocated memory 5.21 GiB is allocated by PyTorch, and 39.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(base_model)

Phi3ForSequenceClassification(
  (model): Phi3Model(
    (embed_tokens): Embedding(32012, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): lora.Linear(
            (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=3072, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (qkv_proj): lora.Linear(
            (base_layer): Linear(in_features=3072, out_featu

In [10]:
from peft import PeftModel
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained("microsoft/Phi-3.5-mini-instruct")
base_model.config.use_cache = False
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, "Colder203/phi3.8b_orm", torch_dtype = torch.float16, device_map = {'': torch.cuda.current_device()})

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]
Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-3.5-mini-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for PeftModelForSequenceClassification:
	size mismatch for base_model.model.score.modules_to_save.default.weight: copying a param with shape torch.Size([1, 3072]) from checkpoint, the shape in current model is torch.Size([2, 3072]).

In [ ]:


def tokenize(sample):
        # question = sample['conversations'][0]['content'].split("Step 1")[0]
        question = sample['prompt']
        samples = []
        for id, ans in enumerate(sample['answers']):
            sam = {}
            # ans = "Step 1" + sample['conversations'][0]['content'].split("Step 1")[1]
            message = [
                {"role":"user", "content":question},
                {"role":"assistant", "content":ans}
            ]
            sam['positive'] = tokenizer.apply_chat_template(
                # message, tokenize=False, add_generation_prompt=False).replace(tokenizer.bos_token, "")
                message, tokenize=False, add_generation_prompt=False)
            tokenized_pos = tokenizer(sam['positive'], truncation=False)
            sam["input_ids_j"] = tokenized_pos["input_ids"]
            sam["attention_mask_j"] = tokenized_pos["attention_mask"]
            if sample['label'][id] == 1:
                sam['label'] = 1
            elif sample['label'][id] == 0:
                sam['label'] = 0
            else:
                assert 1 == 0
            samples.append(sam)
        return samples
    

In [5]:
#evaluating using new dataset
for i, sample in enumerate(new_ds):
    input_ids = torch.tensor(sample["input_ids_j"]).unsqueeze(0).cuda()
    attention_mask = torch.tensor(sample["attention_mask_j"]).unsqueeze(0).cuda()
   
    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)
    probs = torch.sigmoid(output.logits).cpu().numpy()
    label = sample["label"]
    print(probs, label)
    if i == 100:
        break

NameError: name 'new_ds' is not defined

In [ ]:
#evaluating using new dataset
for i, sample in enumerate(new_ds):
    input_ids = torch.tensor(sample["input_ids_j"]).unsqueeze(0).cuda()
    attention_mask = torch.tensor(sample["attention_mask_j"]).unsqueeze(0).cuda()
   
    with torch.no_grad():
        output = model_LLAMA(input_ids=input_ids, attention_mask=attention_mask)
    probs = torch.sigmoid(output.logits).cpu().numpy()
    label = sample["label"]
    print(probs, label)
    if i == 100:
        break